2 - Anomaly detection

2.1 Problem Statement
In this exercise, you will implement an anomaly detection algorithm to detect anomalous behavior in server computers.

The dataset contains two features -

throughput (mb/s) and
latency (ms) of response of each server.
While your servers were operating, you collected  𝑚=307
  examples of how they were behaving, and thus have an unlabeled dataset  {𝑥(1),…,𝑥(𝑚)}
 .

You suspect that the vast majority of these examples are “normal” (non-anomalous) examples of the servers operating normally, but there might also be some examples of servers acting anomalously within this dataset.
You will use a Gaussian model to detect anomalous examples in your dataset.

You will first start on a 2D dataset that will allow you to visualize what the algorithm is doing.
On that dataset you will fit a Gaussian distribution and then find values that have very low probability and hence can be considered anomalies.
After that, you will apply the anomaly detection algorithm to a larger dataset with many dimensions.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from utils import *

%matplotlib inline

ModuleNotFoundError: No module named 'utils'

In [2]:
# Load the dataset
X_train, X_val, y_val = load_data()

NameError: name 'load_data' is not defined

In [ ]:
# Display the first five elements of X_train
print("The first 5 elements of X_train are:\n", X_train[:5])  

In [ ]:
# Display the first five elements of X_val
print("The first 5 elements of X_val are\n", X_val[:5])  

In [ ]:
# Display the first five elements of y_val
print("The first 5 elements of y_val are\n", y_val[:5])  

In [ ]:
print ('The shape of X_train is:', X_train.shape)
print ('The shape of X_val is:', X_val.shape)
print ('The shape of y_val is: ', y_val.shape)

In [ ]:
# Create a scatter plot of the data.
# we used the 'marker' and 'c' parameters
plt.scatter(X_train[:, 0], X_train[:, 1], marker='x', c='b') 

# Set the title
plt.title("The first dataset")
# Set the y-axis label
plt.ylabel('Throughput (mb/s)')
# Set the x-axis label
plt.xlabel('Latency (ms)')
# Set axis range
plt.axis([0, 30, 0, 30])
plt.show()

2.3 Gaussian distribution
To perform anomaly detection, you will first need to fit a model to the data’s distribution.

Given a training set  {𝑥(1),...,𝑥(𝑚)}
  you want to estimate the Gaussian distribution for each of the features  𝑥𝑖
 .

Recall that the Gaussian distribution is given by

𝑝(𝑥;𝜇,𝜎2)=12𝜋𝜎2⎯⎯⎯⎯⎯⎯⎯⎯√exp−(𝑥−𝜇)22𝜎2
 
where  𝜇
  is the mean and  𝜎2
  is the variance.

For each feature  𝑖=1…𝑛
 , you need to find parameters  𝜇𝑖
  and  𝜎2𝑖
  that fit the data in the  𝑖
 -th dimension  {𝑥(1)𝑖,...,𝑥(𝑚)𝑖}
  (the  𝑖
 -th dimension of each example).

 -------------------------------------------------------------------------------------------------------
 
 Exercise 1
Please complete the estimate_gaussian function below to calculate mu (mean for each feature in X) and var (variance for each feature in X).

You can estimate the parameters, ( 𝜇𝑖
 ,  𝜎2𝑖
 ), of the  𝑖
 -th feature by using the following equations. To estimate the mean, you will use:

𝜇𝑖=1𝑚∑𝑗=1𝑚𝑥(𝑗)𝑖
 
and for the variance you will use:
𝜎2𝑖=1𝑚∑𝑗=1𝑚(𝑥(𝑗)𝑖−𝜇𝑖)2

In [3]:

def estimate_gaussian(X): 
    """
    Calculates mean and variance of all features 
    in the dataset
    
    Args:
        X (ndarray): (m, n) Data matrix
    
    Returns:
        mu (ndarray): (n,) Mean of all features
        var (ndarray): (n,) Variance of all features
    """

    m, n = X.shape
    
    mu = np.mean(X, axis=0) 
    var = np.var(X, axis=0) 
            
    return mu, var

print("Mean of each feature:", mu)
print("Variance of each feature:", var)

In [ ]:
# Returns the density of the multivariate normal
# at each data point (row) of X_train
p = multivariate_gaussian(X_train, mu, var)

#Plotting code 
visualize_fit(X_train, mu, var)

2.3.2 Selecting the threshold 𝜖
Now that you have estimated the Gaussian parameters, you can investigate which examples have a very high probability given this distribution and which examples have a very low probability.

The low probability examples are more likely to be the anomalies in our dataset.
One way to determine which examples are anomalies is to select a threshold based on a cross validation set.
In this section, you will complete the code in select_threshold to select the threshold 𝜀
 using the 𝐹1
 score on a cross validation set.

For this, we will use a cross validation set {(𝑥(1)cv,𝑦(1)cv),…,(𝑥(𝑚cv)cv,𝑦(𝑚cv)cv)}
, where the label 𝑦=1
 corresponds to an anomalous example, and 𝑦=0
 corresponds to a normal example.
For each cross validation example, we will compute 𝑝(𝑥(𝑖)cv)
. The vector of all of these probabilities 𝑝(𝑥(1)cv),…,𝑝(𝑥(𝑚cv)cv)
 is passed to select_threshold in the vector p_val.
The corresponding labels 𝑦(1)cv,…,𝑦(𝑚cv)cv
 are passed to the same function in the vector y_val.

Exercise 2
Please complete the select_threshold function below to find the best threshold to use for selecting outliers based on the results from the validation set (p_val) and the ground truth (y_val).

In the provided code select_threshold, there is already a loop that will try many different values of  𝜀
  and select the best  𝜀
  based on the  𝐹1
  score.

You need to implement code to calculate the F1 score from choosing epsilon as the threshold and place the value in F1.

Recall that if an example  𝑥
  has a low probability  𝑝(𝑥)<𝜀
 , then it is classified as an anomaly.

 tp is the number of true positives: the ground truth label says it’s an anomaly and our algorithm correctly classified it as an anomaly.
 fp is the number of false positives: the ground truth label says it’s not an anomaly, but our algorithm incorrectly classified it as an anomaly.
 fn is the number of false negatives: the ground truth label says it’s an anomaly, but our algorithm incorrectly classified it as not being anomalous.

In [ ]:
def select_threshold(y_val, p_val): 
    """
    Finds the best threshold to use for selecting outliers 
    based on the results from a validation set (p_val) 
    and the ground truth (y_val)
    
    Args:
        y_val (ndarray): Ground truth on validation set
        p_val (ndarray): Results on validation set
        
    Returns:
        epsilon (float): Threshold chosen 
        F1 (float):      F1 score by choosing epsilon as threshold
    """ 

    best_epsilon = 0
    best_F1 = 0
    F1 = 0
    
    step_size = (max(p_val) - min(p_val)) / 1000
    
    for epsilon in np.arange(min(p_val), max(p_val), step_size):
    
        predictions = p_val < epsilon

        tp = np.sum((predictions == 1) & (y_val == 1))
        fp = np.sum((predictions == 1) & (y_val == 0))
        fn = np.sum((predictions == 0) & (y_val == 1))

        prec = tp / (tp + fp)
        rec = tp / (tp + fn)

        F1 = 2*prec*rec/(prec+rec)
        
        if F1 > best_F1:
            best_F1 = F1
            best_epsilon = epsilon
        
    return best_epsilon, best_F1

In [ ]:
p_val = multivariate_gaussian(X_val, mu, var)
epsilon, F1 = select_threshold(y_val, p_val)

print('Best epsilon found using cross-validation: %e' % epsilon)
print('Best F1 on Cross Validation Set: %f' % F1)

In [ ]:
# Find the outliers in the training set 
outliers = p < epsilon

# Visualize the fit
visualize_fit(X_train, mu, var)

# Draw a red circle around those outliers
plt.plot(X_train[outliers, 0], X_train[outliers, 1], 'ro',
         markersize= 10,markerfacecolor='none', markeredgewidth=2)

2.4 High dimensional dataset
Now, we will run the anomaly detection algorithm that you implemented on a more realistic and much harder dataset.

In this dataset, each example is described by 11 features, capturing many more properties of your compute servers.

Let's start by loading the dataset.

The load_data() function shown below loads the data into variables X_train_high, X_val_high and y_val_high
_high is meant to distinguish these variables from the ones used in the previous part
We will use X_train_high to fit Gaussian distribution
We will use X_val_high and y_val_high as a cross validation set to select a threshold and determine anomalous vs normal examples

In [ ]:
# load the dataset
X_train_high, X_val_high, y_val_high = load_data_multi()

print ('The shape of X_train_high is:', X_train_high.shape)
print ('The shape of X_val_high is:', X_val_high.shape)
print ('The shape of y_val_high is: ', y_val_high.shape)

In [ ]:
# Apply the same steps to the larger dataset

# Estimate the Gaussian parameters
mu_high, var_high = estimate_gaussian(X_train_high)

# Evaluate the probabilites for the training set
p_high = multivariate_gaussian(X_train_high, mu_high, var_high)

# Evaluate the probabilites for the cross validation set
p_val_high = multivariate_gaussian(X_val_high, mu_high, var_high)

# Find the best threshold
epsilon_high, F1_high = select_threshold(y_val_high, p_val_high)

print('Best epsilon found using cross-validation: %e'% epsilon_high)
print('Best F1 on Cross Validation Set:  %f'% F1_high)
print('# Anomalies found: %d'% sum(p_high < epsilon_high))